# OpenAI 외 도구 호출 에이전트(Tool Calling Agent)
OpenAI 외에도 Anthropic, Google Gemini, Together.ai, Ollama, Mistral과 같은 더 광범위한 공급자 구현을 지원합니다.

이번 챕터에서는 다양한 LLM 을 사용하여 도구 호출 에이전트를 생성하고 실행하는 방법을 살펴보겠습니다.

[LangChain 공식 도큐먼트](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging


logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [3]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

In [4]:
# 도구 정의

@tool
def search_news(query: str) -> List[Dict[str,str]]:
    """Search GoogleNews by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

search_news

StructuredTool(name='search_news', description='Search GoogleNews by input keyword', args_schema=<class 'langchain_core.utils.pydantic.search_news'>, func=<function search_news at 0x000001D6431479C0>)

In [5]:
tools = [search_news]

# Agent 용 프롬프트 생성
- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Tool Calling 을 지원하는 다양한 LLM 목록
실습 진행을 위해서는 아래 내용을 설정해야 합니다.

**~~Anthropic~~**

~~Anthropic API 키 발급 관련~~
~~.env 파일 내 ANTHROPIC_API_KEY 에 발급받은 키를 설정하세요~~

**Gemini**

Gemini API 키 발급 관련
.env 파일 내 GOOGLE_API_KEY 에 발급받은 키를 설정하세요

**Together AI**

Together AI API 키 발급 관련
.env 파일 내 TOGETHER_API_KEY 에 발급받은 키를 설정하세요

**Ollama**

- Ollama Tool Calling 지원 모델 리스트
- 이번 실습에 사용할 llama3.1 모델
- 터미널 창에 ollama pull llama3.1 명령어를 입력하여 모델을 다운로드 받습니다.
- 이전에 Ollama 를 사용하지 않았다면, Ollama 를 참고해 주세요.
- langchain-ollama 설치를 한 뒤 진행해 주세요.

In [7]:
#from langchain_anthropic import ChatAnthropic
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Claude-3-5-sonnet
# claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# Gemini-1.5-pro-latest
#gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
#
# Llama-3.1-70B-Instruct-Turbo
llama = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
)

# Llama-3.1
ollama = ChatOllama(model="llama3.1:latest", temperature=0)

In [8]:
# Agent 생성
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
#claude_agent = create_tool_calling_agent(claude, tools, prompt)
#gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)

In [9]:
# AgentExecuor 실행 후 실행 및 결과 확인
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiakFVX3lxTE81QVlXam1wclZOV3loQndWUjVjU3FKUE9zMzRjbkU5b3RMSjFBaGVfMHBvM28wRDFyd05yMVhQYTVnZy1nVkhGZXBqLTV2Zks5bnFQdG8wVEtBeEJoZllhVkQwQ2d1MENteEE?oc=5', 'content': '오픈AI, 투자 유치 이후 글로벌 확장...뉴욕·파리·싱가포르 등 5곳에 지사 개설 - AI타임스'}, {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE5CX19HNHE1bkF2U1VCeWE5T0ZJQVJIbWZZcHdFMEg2N3FPZ2ZSRnYyazRUSWhGcV9wajFzQl9zRUdxbzJORG1GT0xBN1A1UQ?oc=5', 'content': 'AI 스타트업 보이저엑스, 165억 원 신규 투자 유치 - 매일경제'}, {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE9FdmQ4S2VSMmowbU0yYmVMRkxQNjk5Z2EzRGR5LWJFZVZuYVBvLUhoakYwd0tiM2tGczFfZm16TXJCbDYtbjI4eDg5WUlaVWRzeGx1X2F3?oc=5', 'content': '오펜하이머 "MS,오픈AI 투자 손실 위험"…\'중립\'하향 - 네이트 뉴스'}, {'url': 'https://news.google.com/rss/articles/CBMiYkFVX3lxTE9qZTMzaktQeVE0V19YaEpJZUk1RHVZQnRZbHJ0V2Q2UFYya3hrWDhabjJiR2NlcXlmcjRIMF9JM0x3eXN3VGZKWW9UVEZQbXV3ejhnTl8xMDJhYXdlakdERUN3?oc=

In [10]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

In [11]:
query = (
    "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요."
)

In [12]:
# gpt
execute_agent(gpt, tools, query, "gpt")

[gpt] 결과입니다.
🌟 **AI 투자 소식 모아보기!** 🌟

1️⃣ **오픈AI, 글로벌 확장**  
오픈AI가 투자 유치 후 뉴욕, 파리, 싱가포르 등 5곳에 지사를 개설하며 글로벌 시장으로의 확장을 본격화했습니다! 🌍 [자세히 보기](https://news.google.com/rss/articles/CBMiakFVX3lxTE81QVlXam1wclZOV3loQndWUjVjU3FKUE9zMzRjbkU5b3RMSjFBaGVfMHBvM28wRDFyd05yMVhQYTVnZy1nVkhGZXBqLTV2Zks5bnFQdG8wVEtBeEJoZllhVkQwQ2d1MENteEE?oc=5)

2️⃣ **AI 스타트업, 보이저엑스 165억 원 투자 유치**  
AI 스타트업 보이저엑스가 165억 원의 신규 투자를 유치하며 더 큰 도약을 예고하고 있습니다! 🚀 [자세히 보기](https://news.google.com/rss/articles/CBMiTkFVX3lxTE5CX19HNHE1bkF2U1VCeWE5T0ZJQVJIbWZZcHdFMEg2N3FPZ2ZSRnYyazRUSWhGcV9wajFzQl9zRUdxbzJORG1GT0xBN1A1UQ?oc=5)

3️⃣ **MS, 오픈AI 투자 손실 위험 경고**  
오펜하이머 애널리스트는 MS가 오픈AI에 대한 투자 손실 위험이 있다고 경고하며 중립 평가로 하향 조정했습니다. ⚠️ [자세히 보기](https://news.google.com/rss/articles/CBMiVkFVX3lxTE9FdmQ4S2VSMmowbU0yYmVMRkxQNjk5Z2EzRGR5LWJFZVZuYVBvLUhoakYwd0tiM2tGczFfZm16TXJCbDYtbjI4eDg5WUlaVWRzeGx1X2F3?oc=5)

4️⃣ **메타 부사장, AI는 미래의 근본 기술**  
메타의 부사장은 AI가 근본 기술이라며, 수익보다 미래에 대한 투자가 불가피하다고 밝혔습니다. 🔮 [자세히 보기](https://news.google.com/rss/articles/CBMiYkFVX3

In [13]:
# llama3.1 70B (Together.ai)
execute_agent(
    llama,
    tools,
    "Search AI related news and write it in Instagram post format",
    "llama3.1 70B",
)

[llama3.1 70B] 결과입니다.
Here's a possible Instagram post based on the search results:

"Hey everyone! Did you know that AI is taking over our daily lives? From helping us with our to-do lists to making our daily commutes more efficient, AI is everywhere! 

Check out these latest news articles to learn more about the latest AI trends and developments:

* KT invests 2.4 trillion won in AI and cloud technology over the next 5 years
* The National Assembly inspects the AI Basic Act and R&D budget allocation
* OpenAI expands to Singapore and plans to open an office by the end of the year
* KBank launches an AI quiz challenge for its customers

Stay ahead of the curve and learn more about the latest AI news! #AI #ArtificialIntelligence #Technology"


In [14]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.1(Ollama) 8B")

[llama3.1(Ollama) 8B] 결과입니다.
**AI 투자 뉴스**

* 보이저엑스, 165억 원 신규 투자 유치
* Andrew Ng의 펀드, 인도에서 첫 투자로 AI 헬스케어 회사 Jivi에 100만 달러 투자
* 폭스콘 회장, "AI 투자 붐, 아직 갈 길 있어"고 말합니다.
* ServiceNow와 Zoom, AI 협업 도구 강화
* 모니터코퍼레이션, 830조원대 글로벌 의료 영상 AI 시장에 도전장

#AI #투자 #스타트업 #인공지능


In [15]:
# AgentExecuor 실행 후 실행 및 결과 확인
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=ollama_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)
result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI investment'}`


[{'url': 'https://news.google.com/rss/articles/CBMiaEFVX3lxTFBxRmZQNFc3cXdMOXRJQ0Exa1B2VFdkWDdndDhfbEJwTlNRSERlNGttaXdOaHQza1RrdlRYcHoycEwzWDM2b2tqdFV5cjBzMjd1Yzc0UTE5bXUxQUNlUHJ0UUY5WU9TSGtx0gFsQVVfeXFMT1RjZENyNFlGLThmdzdfWWFQSllrak8xYzZBYUlyRmlEWjlNZ1F4NXBqN3p3UzBPeEFvWXNRdVpyeXVQMUVwM2EtZ3VhbnJkWUoycnVyMXZSdm15V1ZtNGZ1TUc2TlRFVFlFSEZX?oc=5', 'content': 'AI 스타트업 보이저엑스, 165억 원 신규 투자 유치 - 스타트업엔(StartupN)'}, {'url': 'https://news.google.com/rss/articles/CBMi0wFBVV95cUxNSHdBNU1iZmxZdG5sVWJfYkEzdW5XbVdwckdDWkg1aHh5RmcyUE9ROG8xdFRQcnpBWEkxa3BIaV9YdkY5X3MyWUFJcFBqSmlWVEZYdmh5dEFEZWJudW4yTUNhUlBCczI4Vk5XRWw0WUhDN3NjQ2pxREs3Tm9aWkhtNGhodFJwTS1UU1c5ZWM5QnBIb01HeVVaTVZRTkNYUGxyTVh2YmVFUGpDbjRtc1JtT3pCd0Nxb0NZM2dRdjFuekNvUFRkdzN2RDFQWkNqZUNrUk9F?oc=5', 'content': "Andrew Ng's fund makes first India investment with AI healthcare firm Jivi - XM"}, {'url': 'https://news.google.com/rss/articles/CBMi9